# 환경 설정

In [ ]:
# !pip install -q langchain langchain-openai google-search-results

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = 'OPENAI_API_KEY'
os.environ['SERPAPI_API_KEY'] = 'SERPAPI_API_KEY'

# News 데이터 수집 - SerpAPI 활용
- 인증키 발급 필요 (월 100회 무료)
- https://serpapi.com/

In [ ]:
from langchain_community.utilities import SerpAPIWrapper

params = {
    "engine": "google_news",
    "gl": "KR",
    "hl": "ko",
}
search = SerpAPIWrapper(params=params)

search.run("이차전지 산업")

In [ ]:
results = search.run("이차전지 산업")
len(results)

In [ ]:
results[0]

In [ ]:
results[0].keys()

In [ ]:
url = results[0]['link']
url

### LangChain WebBaseLoader 활용

In [ ]:
# Data Loader - 웹페이지 데이터 가져오기
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(url)
docs = loader.load()

docs[0]

In [ ]:
# 뉴스기사에서 본문 텍스트를 추출하는 함수
def extract_longest_text(text):
    # 개행문자(\n\n)를 기준으로 텍스트를 분할
    segments = text.split('\n')
    # 가장 긴 텍스트 조각을 찾음
    longest_segment = max(segments, key=len)
    return longest_segment

text = docs[0].page_content

longest_text = extract_longest_text(text)
print("가장 긴 텍스트:\n", longest_text)

In [ ]:
# SerpAPI 검색 결과를 순회하면서 본문 텍스트를 추출하여 기존 딕셔너리에 추가

from tqdm import tqdm
import time

new_results = []
for result in tqdm(results):
    try:
        url = result['link']
        loader = WebBaseLoader(url)
        docs = loader.load()
        text = docs[0].page_content
        longest_text = extract_longest_text(text)
        result['content'] = longest_text
        new_results.append(result)
    except:
        pass


# 결과 확인
print("새로운 결과 개수:", len(new_results))
new_results[0]

# 판다스 데이터프레임으로 변환

In [ ]:
import pandas as pd
data = pd.DataFrame(new_results)
data.head()

# 뉴스 요약

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Write a concise summary of the following in Korean Hangul (한글):
"{text}"
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

In [ ]:
# 뉴스 본문을 입력으로 사용하여 요약 결과를 생성하는 함수
def summarize_news(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 요약
df_test = data.head(3)
df_test['summary'] = df_test['content'].apply(summarize_news)

df_test

In [ ]:
df_test[['content', 'summary']]

# 키워드 추출

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Please extract 3 key words from the following content in Korean Hangul (한글) and separate them with commas (,):
"{text}"
Key words:"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

In [ ]:
# 뉴스 본문을 입력으로 사용하여 핵심 키워드를 추출하는 함수
def extract_keywords(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 추출
df_test['keywords'] = df_test['content'].apply(extract_keywords)

df_test[['content', 'summary', 'keywords']]

# 뉴스 카테고리 분류

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# prompt
prompt_template = """Based on the following content, please classify the news into the appropriate category and provide the category name in Korean Hangul (한글):
"{text}"
News Category:"""

prompt = PromptTemplate.from_template(prompt_template)

# LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

# output parser
output_parser = StrOutputParser()

# Chain
llm_chain = prompt | llm | output_parser

response = llm_chain.invoke({"text": data['content'][0]})

response

In [ ]:
# 뉴스 본문을 입력으로 사용하여 카테고리를 분류하는 함수
def classify_news_category(content):
    response = llm_chain.invoke({"text": content})
    return response

# 결과 확인 - 테스트를 위해서 첫 3행만 별도로 추출하여 카테고리 분류
df_test['category'] = df_test['content'].apply(classify_news_category)

df_test[['content', 'summary', 'keywords', 'category']]

### 키워드에서 한글만 추출

In [ ]:
import re
korean_word = re.findall(r'[가-힣]+', df_test['category'][0])[0]
korean_word

In [ ]:
# 람다 함수를 사용하여 카테고리에서 한글 단어만 추출 
df_test['category'].apply(lambda x: re.findall(r'[가-힣]+', x)[0])

In [ ]:
df_test['category'] = df_test['category'].apply(lambda x: re.findall(r'[가-힣]+', x)[0])
df_test[['content', 'summary', 'keywords', 'category']]